In [79]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, FloatType
from pyspark.sql import SQLContext
from pyspark.sql.functions import col
from pyspark.sql.functions import sum, avg

In [2]:
spark = SparkContext(master='local', appName='Dataframes')
sqlContext = SQLContext(spark)

In [3]:
!ls files

deporte.csv	 deportistaError.csv  modelo_relacional.jpg
deportista2.csv  evento.csv	      paises.csv
deportista.csv	 juegos.csv	      resultados.csv


In [4]:
#aca cargo la ruta del archivo
path = "files/"

In [5]:
equiposOlimpicosRDD = spark.textFile(path+'paises.csv') \
    .map(lambda line : line.split(','))

In [6]:
#sc.stop() #stop detiene el contexto

In [7]:
deportistaOlimpicoRDD = spark.textFile(path+'deportista.csv').map(lambda l: l.split(','))
deportistaOlimpicoRDD2 = spark.textFile(path+'deportista2.csv').map(lambda l: l.split(','))

In [8]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.union(deportistaOlimpicoRDD2)

In [9]:
deportistaOlimpicoRDD.count()

135572

In [10]:
def eliminaEncabezado(indice, interador):
    return iter(list(interador)[1:])

In [11]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.mapPartitionsWithIndex(eliminaEncabezado)

In [12]:
deportistaOlimpicoRDD = deportistaOlimpicoRDD.map(lambda l: 
    (
    int(l[0]),
    l[1],
    int(l[2]),
    int(l[3]),
    int(l[4]),
    float(l[5]),
    int(l[6])
    )
)

In [13]:
schema = StructType([
    StructField('deportista_id', IntegerType(), False),
    StructField('nombre', StringType(), False),
    StructField('genero', IntegerType(), False),
    StructField('edad', IntegerType(), False),
    StructField('altura', IntegerType(), False),
    StructField('peso', FloatType(), False),
    StructField('equipo_id', IntegerType(), False )
])

In [14]:
deportistaOlimpicoDF = sqlContext.createDataFrame(deportistaOlimpicoRDD, schema)

In [15]:
deportistaOlimpicoDF.show(3)

+-------------+-------------------+------+----+------+----+---------+
|deportista_id|             nombre|genero|edad|altura|peso|equipo_id|
+-------------+-------------------+------+----+------+----+---------+
|            1|          A Dijiang|     1|  24|   180|80.0|      199|
|            2|           A Lamusi|     1|  23|   170|60.0|      199|
|            3|Gunnar Nielsen Aaby|     1|  24|     0| 0.0|      273|
+-------------+-------------------+------+----+------+----+---------+
only showing top 3 rows



In [16]:
evento_schema = StructType([
    StructField('evento_id', IntegerType(), False),
    StructField('evento', StringType(), False),
    StructField('deporte_id', IntegerType(), False)
])

In [17]:
eventoDF = sqlContext.read.format("csv").\
        option("header", True).\
        schema(evento_schema).\
        load(path+"evento.csv")

In [18]:
eventoDF.show(4)

+---------+--------------------+----------+
|evento_id|              evento|deporte_id|
+---------+--------------------+----------+
|        1|Basketball Men's ...|         1|
|        2|Judo Men's Extra-...|         2|
|        3|Football Men's Fo...|         3|
|        4|Tug-Of-War Men's ...|         4|
+---------+--------------------+----------+
only showing top 4 rows



In [19]:
paises_schema = StructType([
    StructField("paises_id", IntegerType(), False),
    StructField("equipo", StringType(), False), 
    StructField("sigla", StringType(), False)
])


In [20]:
paisesDF = sqlContext.read.format("csv").\
        option("header", True).\
        schema(paises_schema).\
        load(path+"paises.csv")


In [21]:
paisesDF.show(4)

+---------+--------------------+-----+
|paises_id|              equipo|sigla|
+---------+--------------------+-----+
|        1|         30. Februar|  AUT|
|        2|A North American ...|  MEX|
|        3|           Acipactli|  MEX|
|        4|             Acturus|  ARG|
+---------+--------------------+-----+
only showing top 4 rows



In [22]:
resultados_schema = StructType([
    StructField("resultado_id", IntegerType(), False),
    StructField("medalla", StringType(), False),
    StructField("deportista_id", IntegerType(), False),
    StructField("juego_id", IntegerType(), False),
    StructField("evento_id", IntegerType(), False)
])


In [23]:
resultadosDF = sqlContext.read.format("csv").\
            option("header", True).\
            schema(resultados_schema).\
            load(path+"resultados.csv")


In [24]:
resultadosDF.show(4)

+------------+-------+-------------+--------+---------+
|resultado_id|medalla|deportista_id|juego_id|evento_id|
+------------+-------+-------------+--------+---------+
|           1|     NA|            1|      39|        1|
|           2|     NA|            2|      49|        2|
|           3|     NA|            3|       7|        3|
|           4|   Gold|            4|       2|        4|
+------------+-------+-------------+--------+---------+
only showing top 4 rows



In [25]:
deportes_schema = StructType([
    StructField("deporte_id", IntegerType(), False),
    StructField("deporte", StringType(), False)
])

In [26]:
deportesDF = sqlContext.read.format("csv").\
            option("header", True).\
            schema(deportes_schema).\
            load(path+"deporte.csv")

In [27]:
deportesDF.show(2)

+----------+----------+
|deporte_id|   deporte|
+----------+----------+
|         1|Basketball|
|         2|      Judo|
+----------+----------+
only showing top 2 rows



In [28]:
deportesDF.printSchema()

root
 |-- deporte_id: integer (nullable = true)
 |-- deporte: string (nullable = true)



In [29]:
deportistaOlimpicoDF.printSchema()

root
 |-- deportista_id: integer (nullable = false)
 |-- nombre: string (nullable = false)
 |-- genero: integer (nullable = false)
 |-- edad: integer (nullable = false)
 |-- altura: integer (nullable = false)
 |-- peso: float (nullable = false)
 |-- equipo_id: integer (nullable = false)



In [30]:
#eliminanado y renombrando columnas
deportistaOlimpicoDF = deportistaOlimpicoDF.withColumnRenamed('genero','sexo').drop('altura')

In [31]:
deportistaOlimpicoDF.printSchema()

root
 |-- deportista_id: integer (nullable = false)
 |-- nombre: string (nullable = false)
 |-- sexo: integer (nullable = false)
 |-- edad: integer (nullable = false)
 |-- peso: float (nullable = false)
 |-- equipo_id: integer (nullable = false)



In [32]:
deportistaOlimpicoDF = deportistaOlimpicoDF.select('deportista_id', 'nombre', col('edad').alias('edadAlJugar'), 'equipo_id')

In [33]:
deportistaOlimpicoDF.show(5)

+-------------+--------------------+-----------+---------+
|deportista_id|              nombre|edadAlJugar|equipo_id|
+-------------+--------------------+-----------+---------+
|            1|           A Dijiang|         24|      199|
|            2|            A Lamusi|         23|      199|
|            3| Gunnar Nielsen Aaby|         24|      273|
|            4|Edgar Lindenau Aabye|         34|      278|
|            5|Christine Jacoba ...|         21|      705|
+-------------+--------------------+-----------+---------+
only showing top 5 rows



In [35]:
deportistaOlimpicoDF = deportistaOlimpicoDF.filter(deportistaOlimpicoDF.edadAlJugar != 0)

In [36]:
deportistaOlimpicoDF.sort('edadAlJugar').show(5)

+-------------+--------------------+-----------+---------+
|deportista_id|              nombre|edadAlJugar|equipo_id|
+-------------+--------------------+-----------+---------+
|        71691|  Dimitrios Loundras|         10|      333|
|        52070|        Etsuko Inada|         11|      514|
|        40129|    Luigina Giavotti|         11|      507|
|        37333|Carlos Bienvenido...|         11|      982|
|        47618|Sonja Henie Toppi...|         11|      742|
+-------------+--------------------+-----------+---------+
only showing top 5 rows



In [37]:
juego_schema = StructType([
    StructField("juego_id", IntegerType(), False),
    StructField("anio", StringType(), False),
    StructField("temporada", StringType(), False),
    StructField("ciudad", StringType(), False)
])

In [39]:
juegoDF = sqlContext.read.format("csv").\
            option("header", True).\
            schema(juego_schema).\
            load(path+"juegos.csv")

In [41]:
juegoDF.show(4)

+--------+-----------+---------+------+
|juego_id|       anio|temporada|ciudad|
+--------+-----------+---------+------+
|       1|1896 Verano|     1896|Verano|
|       2|1900 Verano|     1900|Verano|
|       3|1904 Verano|     1904|Verano|
|       4|1906 Verano|     1906|Verano|
+--------+-----------+---------+------+
only showing top 4 rows



In [42]:
deportesDF.show(5)

+----------+-------------+
|deporte_id|      deporte|
+----------+-------------+
|         1|   Basketball|
|         2|         Judo|
|         3|     Football|
|         4|   Tug-Of-War|
|         5|Speed Skating|
+----------+-------------+
only showing top 5 rows



In [58]:
deportistaOlimpicoDF.join(resultadosDF, deportistaOlimpicoDF.deportista_id == resultadosDF.deportista_id, 'left') \
.join(juegoDF, juegoDF.juego_id == resultadosDF.juego_id, 'left') \
.join(eventoDF, eventoDF.evento_id == resultadosDF.evento_id, 'left') \
.select(deportistaOlimpicoDF.nombre ,'edadAlJugar','medalla', col('anio').alias('año de juego'), eventoDF.evento.alias('nombre de desciplina')).show()

+--------------------+-----------+-------+-------------+--------------------+
|              nombre|edadAlJugar|medalla| año de juego|nombre de desciplina|
+--------------------+-----------+-------+-------------+--------------------+
|           A Dijiang|         24|     NA|  1992 Verano|Basketball Men's ...|
|            A Lamusi|         23|     NA|  2012 Verano|Judo Men's Extra-...|
| Gunnar Nielsen Aaby|         24|     NA|  1920 Verano|Football Men's Fo...|
|Edgar Lindenau Aabye|         34|   Gold|  1900 Verano|Tug-Of-War Men's ...|
|Christine Jacoba ...|         21|     NA|1994 Invierno|Speed Skating Wom...|
|Christine Jacoba ...|         21|     NA|1994 Invierno|Speed Skating Wom...|
|Christine Jacoba ...|         21|     NA|1992 Invierno|Speed Skating Wom...|
|Christine Jacoba ...|         21|     NA|1992 Invierno|Speed Skating Wom...|
|Christine Jacoba ...|         21|     NA|1988 Invierno|Speed Skating Wom...|
|Christine Jacoba ...|         21|     NA|1988 Invierno|Speed Sk

In [61]:
resultadosDF.filter(resultadosDF.medalla != 'NA') \
    .join(deportistaOlimpicoDF, deportistaOlimpicoDF.deportista_id == resultadosDF.deportista_id, 'left') \
        .join(paisesDF, paisesDF.paises_id == deportistaOlimpicoDF.equipo_id, 'left') \
            .select('medalla', 'equipo', 'sigla') \
                .sort( col('sigla').desc()).show()

+-------+--------+-----+
|medalla|  equipo|sigla|
+-------+--------+-----+
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
| Bronze|Zimbabwe|  ZIM|
| Silver|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
|   Gold|Zimbabwe|  ZIM|
+-------+--------+-----+
only showing top 20 rows



In [70]:
medallistaXAnio = deportistaOlimpicoDF \
    .join(
        resultadosDF, 
        deportistaOlimpicoDF.deportista_id == resultadosDF.deportista_id, 
        "left"
    ) \
    .join(
        juegoDF, 
        juegoDF.juego_id == resultadosDF.juego_id, 
        "left"
    ) \
    .join(
        paisesDF, 
        deportistaOlimpicoDF.equipo_id == paisesDF.paises_id, 
        "left"
    ) \
    .join(
        eventoDF, 
        eventoDF.evento_id == resultadosDF.evento_id, 
        "left"
    ) \
    .join(
        deportesDF, 
        eventoDF.deporte_id == deportesDF.deporte_id, 
        "left"
    ) \
    .select(
        "sigla",
        "anio",
        "medalla",
        eventoDF.evento.alias("nombre subdisciplina"),
        deportesDF.deporte.alias("nombre disciplina"),
        deportistaOlimpicoDF.nombre    
    )


In [71]:
medallistaXAnio.show(10)

+-----+-------------+-------+--------------------+-----------------+--------------------+
|sigla|         anio|medalla|nombre subdisciplina|nombre disciplina|              nombre|
+-----+-------------+-------+--------------------+-----------------+--------------------+
|  CHN|  1992 Verano|     NA|Basketball Men's ...|       Basketball|           A Dijiang|
|  CHN|  2012 Verano|     NA|Judo Men's Extra-...|             Judo|            A Lamusi|
|  DEN|  1920 Verano|     NA|Football Men's Fo...|         Football| Gunnar Nielsen Aaby|
|  SWE|  1900 Verano|   Gold|Tug-Of-War Men's ...|       Tug-Of-War|Edgar Lindenau Aabye|
|  NED|1994 Invierno|     NA|Speed Skating Wom...|    Speed Skating|Christine Jacoba ...|
|  NED|1994 Invierno|     NA|Speed Skating Wom...|    Speed Skating|Christine Jacoba ...|
|  NED|1992 Invierno|     NA|Speed Skating Wom...|    Speed Skating|Christine Jacoba ...|
|  NED|1992 Invierno|     NA|Speed Skating Wom...|    Speed Skating|Christine Jacoba ...|
|  NED|198

In [73]:
medallistaXAnio2 = medallistaXAnio.filter(medallistaXAnio.medalla != 'NA') \
    .sort('anio') \
        .groupBy('sigla', 'anio','nombre subdisciplina') \
            .count()

In [74]:
medallistaXAnio2.printSchema()

root
 |-- sigla: string (nullable = true)
 |-- anio: string (nullable = true)
 |-- nombre subdisciplina: string (nullable = true)
 |-- count: long (nullable = false)



In [80]:
medallistaXAnio2.groupBy('sigla', 'anio') \
    .agg(sum('count').alias('total de me dallas'), avg('count').alias('medalla promedio')).show()

+-----+-------------+------------------+------------------+
|sigla|         anio|total de me dallas|  medalla promedio|
+-----+-------------+------------------+------------------+
|  NED|1992 Invierno|                 4|1.3333333333333333|
|  BEL|  2000 Verano|                 7|               1.4|
|  MAS|  2012 Verano|                 2|               1.0|
|  MGL|  2008 Verano|                 5|              1.25|
|  SWE|  1976 Verano|                10|               2.0|
|  SUI|2014 Invierno|                29|3.2222222222222223|
|  ETH|  2004 Verano|                 7|              1.75|
|  AUT|  1928 Verano|                 5|              1.25|
|  SYR|  1984 Verano|                 1|               1.0|
|  ITA|  1996 Verano|                69| 2.225806451612903|
|  THA|  2008 Verano|                 4|               1.0|
|  URS|1984 Invierno|                56|               2.8|
|  DEN|  1896 Verano|                 6|               1.0|
|  GRN|  2016 Verano|                 1|